In [1]:
!pip install fuzzywuzzy 
!pip install  python-Levenshtein

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144791 sha256=46e7ff6d85c4e013ea0d2211fdbc265686fae766af380aff6075330bb25e4fdb
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [2]:
#This notebook presents the final result with detecting the duplicates
#and returning the percentage of dupes and the clean  dataframe 
from fuzzywuzzy import fuzz
import pandas as pd
import time

In [3]:
def detect_duplicates(df):
    
    """Return the cleaned dataframe from duplicates and the percentage of duplicates
    found
    :param1 : the dataframe
    """
    #The three criteria we choose to detect a duplicate
    df1=df["given_name"] 
    df2=df["surname"]
    df3=df["phone_number"]
    #A list containing the row index of the duplicates
    dup_idx = []
    #Copy of the dataframe so we can compare it to the original
    df_clear=df.copy()
  

    for i in range(len(df1.values)-1):
        
        for j in range(i+1,len(df1.values)):
            
            if fuzz.token_sort_ratio(df1.values[i], df1.values[j]) >= 75 and fuzz.token_sort_ratio(df2.values[i], df2.values[j]) >= 75 and  df3.values[i]==df3.values[j]:
                
                dup_idx.append(df1.index[j])

    df_clear = df.drop(df.index[dup_idx])
    percentage = round((len(dup_idx)/len(df))*100,2)
   
      
    return  percentage,df_clear

In [5]:
#Reading our csv and dataframe

df_patient = pd.read_csv("df_dedup_clean.csv")
df_patient


,patient_id,given_name,surname,street_number,address_1,phone_number,address_2,postcode_updated,state_updated,suburb_updated,dob_updated,age_final
0,221958,matisse,clarke,13.0,rene street,08 86018809,westella,2527,nsw,ellenbrook,1971,49
1,771155,joshua,elrick,23.0,andrea place,02 97793152,foxdown,2074,nsw,east preston,1912,108
2,231932,alice,conboy,35.0,mountain circuit,02 20403934,None,2305,nsw,prospect,1981,39
3,465838,sienna,craswell,39.0,cumberlegeicrescent,02 62832318,jodane,3620,vic,henty,1984,36
4,359178,joshua,bastiaans,144.0,lowrie street,03 69359594,None,4051,qld,campbell town,1934,86
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,368617,abby,None,19.0,john cleland crescent,02 92489054,glenmore,2290,nsw,boyne island,1957,63
19996,796658,dale,priest,19.0,bellchambers crescent,07 20549476,None,3033,vic,hoppers crossing,1959,61
19997,511885,alessia,mawlai,4.0,miller street,03 83466737,None,3616,vic,smithfield,1994,26
19998,985932,hannah,mason,1.0,halligan place,04 41125514,None,2443,nsw,beaumaris,1914,106


In [6]:
#Detect and duplicates 
#We choose the name, surname and phone_number as criteria for detecting the dupes
#generally speaking they're the most "trustworthy" data and allows to really filter
#the persons. 

start = time.time()
pourcentage,df_clear = detect_duplicates(df_patient)
end = time.time()
print(end - start)

2124.5248811244965


In [7]:
print("Time execution is {:.1f} min".format(round((end - start)/60),1))

Time execution is 35.0 min


In [8]:
print("Percentage of duplicates is {}%".format(pourcentage))

Percentage of duplicates is 7.43%


In [9]:
#Display new clean dataframe
df_clear

,patient_id,given_name,surname,street_number,address_1,phone_number,address_2,postcode_updated,state_updated,suburb_updated,dob_updated,age_final
0,221958,matisse,clarke,13.0,rene street,08 86018809,westella,2527,nsw,ellenbrook,1971,49
1,771155,joshua,elrick,23.0,andrea place,02 97793152,foxdown,2074,nsw,east preston,1912,108
2,231932,alice,conboy,35.0,mountain circuit,02 20403934,None,2305,nsw,prospect,1981,39
3,465838,sienna,craswell,39.0,cumberlegeicrescent,02 62832318,jodane,3620,vic,henty,1984,36
4,359178,joshua,bastiaans,144.0,lowrie street,03 69359594,None,4051,qld,campbell town,1934,86
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,368617,abby,None,19.0,john cleland crescent,02 92489054,glenmore,2290,nsw,boyne island,1957,63
19996,796658,dale,priest,19.0,bellchambers crescent,07 20549476,None,3033,vic,hoppers crossing,1959,61
19997,511885,alessia,mawlai,4.0,miller street,03 83466737,None,3616,vic,smithfield,1994,26
19998,985932,hannah,mason,1.0,halligan place,04 41125514,None,2443,nsw,beaumaris,1914,106


In [10]:
print("{} of rows were deleted".format(20000-len(df_clear)))

1045 of rows were deleted


In [11]:
#Save the new clean dataframe 
df_clear.to_csv('df_patient_clear_final_f.csv',index=False)


In [36]:
#We define the following test in the test.py file
def test_detect_duplicates():

    """This test function assure that duplicates were removed
       We tested it on the file df_patient_dup which contains
       all the dupes from the first 10 000 entries of our dataframe

    """
    assert detect_duplicates(df)!=(0,df)


In [33]:
!python -m pytest test.py

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

test.py .                                                                [100%]

=========================== 1 passed in 2.53 seconds ===========================


In [38]:
#These are complementary/optionnal tests
import pandas.api.types as ptypes
from pandas._testing import assert_frame_equal

        
def test_verify_dataframe(param1,param2,param3,param4=None):
    
    """Assert that the dataframe given as criteria are Object types 
    :param param1 : the percentage of duplicates, 
    :param param2 : the original dataframe
    :param param3 : the cleaned dataframe
    :param param4 : the expected dataframe (if available)

    """
    
  
    if param1 !=0:
        assert len(param3)<len(param2)
    else:
        
        assert_frame_equal(param3, param2) #if there is no duplicates 
  
    if param4 is not None:
        
        assert_frame_equal(param4, param3)
    


def test_verify_dtypes(param1,param2,param3):
    """Assert that the columns given as criteria are Object types 
    :param param1 : the dataframe, 
    :param param2 : the selected criteria column of the dataframe
    :param param3 : the selected criteria column of the dataframe

    """
  
    assert ptypes.is_string_dtype(param1[param2]) and ptypes.is_string_dtype(param1[param3])
  

In [39]:
test_verify_dataframe(pourcentage,df_patient,df_clear,param4=df_clear)
test_verify_dtypes(df_patient,"given_name","surname")